<a href="https://colab.research.google.com/github/urieliram/analog/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
for item in 'ABCDEFG':
print(item)

In [ ]:
print('Line 2')